In [1]:
import torch
import torch.nn as nn

from transformers import AutoTokenizer
from tokenizers import Tokenizer
from datasets import load_dataset
from coop.models import Optimus
from src.base import BaseConfig, switch_dict_tensor_device

/home/ubuntu/.local/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

class VAEConfig(BaseConfig):
    latent_dim: int = 512
    label_for_pad_id: int = -100
    max_seq_len: int = 128
    dataset_name: str = "yelp_polarity"
    device: str = 'cpu'


config = VAEConfig()
vae = Optimus(
    config.latent_dim,
    config.label_for_pad_id,
    50256,
    50256
).eval()

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of OptimusDecoder were not initialized from the model checkpoint at gpt2 and are newly initialized: ['linear_mem.weight', 'linear_emb.weight']
Y

In [3]:
vae.load_state_dict(torch.load("vae.pt", map_location='cpu'))

<All keys matched successfully>

In [4]:
enc_tok = AutoTokenizer.from_pretrained("bert-base-cased")
dec_tok = AutoTokenizer.from_pretrained("gpt2")

In [6]:
texts = """
this food is amazing!!
this food is delicious
this food is disgusting
omg what a wonderful restaurant 
""".strip().split("\n")
for text in texts:
    latent = vae(src=enc_tok(text, return_tensors="pt")).q.mean
    print(text, "->", dec_tok.decode(vae.generate(latent, max_tokens=64, min_length=4, no_repeat_ngram_size=2, num_beams=5)[0]))

this food is amazing!! -> <|endoftext|> really good service sucks! I've been going to find this place was very disappointed. I had a few months ago since I was told that I went to the first time I decided to give it a couple weeks ago I asked if it was seated seated at least 10 minutes later I wanted to go back again again I
this food is delicious -> <|endoftext|> really good service sucks! I've been going to find this place was very disappointed. I had a few months ago since I was told that I went to the first time I decided to give it a couple weeks ago I asked if it was seated seated at least 10 minutes later I wanted to go back again again I
this food is disgusting -> <|endoftext|> really good service sucks! I've been going to find this place was very disappointed. I had a few months ago since I was told that I went to the first time I decided to give it a couple weeks ago I asked if it was seated seated at least 10 minutes later I wanted to go back again again I


KeyboardInterrupt: 